Load testfiles

In [1]:
# Import libraries
# !pip install lime
# os.chdir("../")
import os
import pandas as pd
import pickle
from sklearn.ensemble import RandomForestClassifier
import matplotlib.pyplot as plt
import random
# import lime
from lime import lime_tabular
import shap

import json
from typing import Union
from datetime import datetime as dt
import numpy as np
from compactor.ActivatedCompactor import ActivatedCompactor
from compactor.MaxCompactor import MaxCompactor
from compactor.MaxNonActivatedCompactor import MaxNonActivatedCompactor

IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html


In [2]:
PROCESSED_DATA = "/storage/scratch/e17-fyp-xai/projects/e17-4yp-using-machine-learning-in-high-stake-settings/code/processed_data/processed_final_data_latest.csv"

In [3]:
raw_data = pd.read_csv(PROCESSED_DATA)

In [4]:
raw_data.head()

,Unnamed: 0,Project ID,Project Posted Date,Project Type,Project Posted Month,Project Subject Category Tree,Project Cost,Project Subject Subcategory Tree,Project Grade Level Category,Project Resource Category,...,Label,Teacher Success Rate,Teacher Success Rate Imputed,School City Success Rate,School City Success Rate Imputed,School Success Rate,School Success Rate Imputed,School County Success Rate,School County Success Rate Imputed,Project Count in State
0,0,699d6d657a8cd2a83ba09ee8bbb4bacf,2015-08-23,Teacher-Led,8,"Applied Learning, Special Needs",511.29,"Other, Special Needs",Grades PreK-2,Other,...,0,0.0,1,0.726070,0,1.0000,0,0.753059,0,3584
1,2,9712349367ff50b9eab00445594a25cd,2016-04-02,Teacher-Led,4,Literacy & Language,189.58,"ESL, Literacy",Grades PreK-2,Books,...,0,0.0,1,0.584906,0,0.3125,0,0.676056,0,7070
2,4,baa270ad21f432dcf6943cfe0cf51313,2016-09-16,Teacher-Led,9,Health & Sports,826.89,Health & Wellness,Grades 6-8,Supplies,...,0,0.0,1,0.621951,0,0.0000,0,0.658787,0,8875
3,6,cf54d3a016f6f715f5cea46f0aba15e0,2015-11-08,Teacher-Led,11,Literacy & Language,653.28,Literacy,Grades PreK-2,Books,...,0,0.0,1,0.600000,0,0.0000,1,0.694737,0,876
4,7,05970d5f4232634dee18e8e92a19c206,2014-08-01,Teacher-Led,8,Math & Science,690.67,Mathematics,Grades 3-5,Supplies,...,1,0.0,1,0.750000,0,0.0000,1,0.750000,0,627


In [5]:
raw_data[raw_data["Project ID"] == "cdc83b2eb031d999c123535bf09d24a0" ]

,Unnamed: 0,Project ID,Project Posted Date,Project Type,Project Posted Month,Project Subject Category Tree,Project Cost,Project Subject Subcategory Tree,Project Grade Level Category,Project Resource Category,...,Label,Teacher Success Rate,Teacher Success Rate Imputed,School City Success Rate,School City Success Rate Imputed,School Success Rate,School Success Rate Imputed,School County Success Rate,School County Success Rate Imputed,Project Count in State
314336,603375,cdc83b2eb031d999c123535bf09d24a0,2016-11-14,Teacher-Led,11,Math & Science,3334.51,Mathematics,Grades PreK-2,Technology,...,1,0.4,0,0.388889,0,0.285714,0,0.614035,0,682


In [6]:
!ls

 compactor		       helper.py
 config.py		       kernelshap_explainer.py
 data_processer.ipynb	       lime_explainer.py
 data_processor.py	       main.py
 disagreement.ipynb	       model_outputs
 disagreement.py	       model_parameters
 disagreement_v2-Copy1.ipynb   __pycache__
 disagreement_v2.ipynb	       README.md
 eda.py			       requirements.txt
 environment.yml	       run_explanations.py
 Feature_Engineering.ipynb     Sanity_checks.ipynb
 Feature_Engineering.py        temporal_features.py
 feature_engineer.py	       test.png
'Global Vs lime.ipynb'	       treeshap_explainer.py


In [7]:
os.chdir("/storage/scratch/e17-fyp-xai/projects/thanujan/e17-4yp-using-machine-learning-in-high-stake-settings/code")
import os
import feature_engineer as fe 
import data_processor as dp
import config
from helper import (
    save_model,
    load_model,
    create_dirs,
    create_classification_models,
    create_logistic_regression_parameters,
    create_random_forest_parameters,
    create_xgb_classifier_parameters,
    log_intermediate_output_to_file,
    filter_dataset_by_date)
import temporal_features as tmpf
import config

In [8]:
data = dp.set_data_types_to_datetime(raw_data, ["Project Posted Date"])
data = filter_dataset_by_date(data)
data_1 = dp.encode_data(data, config.CATEGORICAL_COLS)
data_folds = fe.split_data_folds(data_1)


-----
train    2016-01-07 - 2016-07-05
test     2016-11-02 - 2016-12-02
(238834, 578)
(92313, 577)
(92313, 576)
(92313, 1)
(32134, 578)
(20079, 577)
(20079, 576)
(20079, 576)
Training set shape =  (92313, 576)
Testing set shape =  (20079, 576)
-----
-----
train    2015-09-09 - 2016-03-07
test     2016-07-05 - 2016-08-04
(314310, 578)
(105920, 577)
(105920, 576)
(105920, 1)
(146069, 578)
(18750, 577)
(18750, 576)
(18750, 576)
Training set shape =  (105920, 576)
Testing set shape =  (18750, 576)
-----
-----
train    2015-05-12 - 2015-11-08
test     2016-03-07 - 2016-04-06
(370969, 578)
(99939, 577)
(99939, 576)
(99939, 1)
(207731, 578)
(23932, 577)
(23932, 576)
(23932, 576)
Training set shape =  (99939, 576)
Testing set shape =  (23932, 576)
-----
-----
train    2015-01-12 - 2015-07-11
test     2015-11-08 - 2015-12-08
(416306, 578)
(61637, 577)
(61637, 576)
(61637, 1)
(270016, 578)
(18400, 577)
(18400, 576)
(18400, 576)
Training set shape =  (61637, 576)
Testing set shape =  (18400, 576)

In [9]:
# # for data in :
# #     if data=="cdc83b2eb031d999c123535bf09d24a0":
# #         print(data)

# data_folds[0]["x_test"][data_folds[0]["x_test"]["Project ID"] == "cdc83b2eb031d999c123535bf09d24a0"].columns == 

In [10]:
from disagreement import Disagreement
import json

In [11]:
!ls 

 compactor		       helper.py
 config.py		       kernelshap_explainer.py
 data_processer.ipynb	       lime_explainer.py
 data_processor.py	       main.py
 disagreement.ipynb	       model_outputs
 disagreement.py	       model_parameters
 disagreement_v2-Copy1.ipynb   __pycache__
 disagreement_v2.ipynb	       README.md
 eda.py			       requirements.txt
 environment.yml	       run_explanations.py
 Feature_Engineering.ipynb     Sanity_checks.ipynb
 Feature_Engineering.py        temporal_features.py
 feature_engineer.py	       test.png
'Global Vs lime.ipynb'	       treeshap_explainer.py


In [12]:
explanations = json.load(open("/storage/scratch/e17-fyp-xai/projects/e17-4yp-using-machine-learning-in-high-stake-settings/code/model_outputs_5/xai_results/all_exp.json"))

In [13]:
!ls /storage/scratch/e17-fyp-xai/projects/e17-4yp-using-machine-learning-in-high-stake-settings/

 code
 code_base_II
 docs
 Documents
 EDA_Projects.ipynb
 fold_data
'Model explaination for top 50 and bottom 50 -- lime.ipynb'
 nohup.out
 output_Xai_latest.out
 output_Xai_latest_sep_18.out
 output_Xai_latest_sep_19.out
 output_Xai.out
'ratio_of averages.csv'
 README.md
 Sanity_checks_2.ipynb
 Sanity_checks.ipynb
 thanujan.py
 Untitled.ipynb
 XAI_Pipeline.ipynb
 XAI_Pipeline_latest.ipynb
 XAI_Pipeline_new.ipynb
 XAI_Pipeline_new_latest_out.ipynb
 XAI_Pipeline_new_out.ipynb
 XAI_Pipeline_result_16_sep.ipynb
 XAI_Pipeline_sep_18.ipynb
 XAI_Pipeline_sep_19.ipynb
 XAI_Pipeline_sep_21_top_500_bottom_500.ipynb
 XAI_Pipeline_sep_22_top_500_bottom_500_2.ipynb
 XAI_Pipeline_sep_22_top_500_bottom_500.ipynb
 XAI_Pipeline_sep_23_latest.ipynb
 XAI_Pipeline_sep_23_latest_output.ipynb
 XAI_Pipeline_test.ipynb
 xai_results
 xai_test_results


In [14]:
explanations.keys()

dict_keys(['fold5', 'fold4', 'fold3', 'fold2', 'fold1', 'fold0'])

In [15]:
explanations["fold0"].keys()

dict_keys(['lime', 'treeshap'])

In [16]:
explanations["fold0"]["lime"].keys()

dict_keys(['top', 'bottom'])

Projects in the lime top explanations

In [17]:
list(explanations["fold0"]["lime"]["top"].keys())[:10]

['cdc83b2eb031d999c123535bf09d24a0',
 '203c08ca3b1e42a9ba07c93eac4c20cc',
 'd1fb6298685f8afb718cc48543e2e244',
 'a2d940be8dc020e99c4f23fa979994bf',
 'f82498e55e15c90643967f8a38ddd98c',
 '53421ba867de46ff359de40e9753711e',
 '4b972ebb44d4ffc7516c8ff160301ef3',
 '243ac2a985a19d4df09c21541494dcae',
 'd1715d9dc331bee79ee781ff26684f8f',
 '6e87bf75a7ec7e474e90aef4188a7007']

Projects in the lime bottom explanations

In [18]:
list(explanations["fold0"]["lime"]["bottom"].keys())[:10]

['9922bc88316d50dc9fec75ace1f6428d',
 'cfbd2d9e5b4e27068f9463b029a3d777',
 '93b9f3988e3941f7acf43b2917e823fe',
 '975a0183c407402eb3fc4b32fe2df0a5',
 '77d5e0dc0c59a21425cc417f43266b99',
 'b53c6be1e99af4ad3b782cbbbc14c209',
 'd0cfa36c791f4d83123c4b9853983418',
 'bca108ff397e3f4461a6dbe966996eae',
 '1e45580a189dcba2b0a581aea5690850',
 '914ed5ecded471bcb3922ccf422d9265']

Projects in the treeshap top explanations

In [19]:
list(explanations["fold0"]["treeshap"]["top"].keys())[:10]

['cdc83b2eb031d999c123535bf09d24a0',
 '203c08ca3b1e42a9ba07c93eac4c20cc',
 'd1fb6298685f8afb718cc48543e2e244',
 'a2d940be8dc020e99c4f23fa979994bf',
 'f82498e55e15c90643967f8a38ddd98c',
 '53421ba867de46ff359de40e9753711e',
 '4b972ebb44d4ffc7516c8ff160301ef3',
 '243ac2a985a19d4df09c21541494dcae',
 'd1715d9dc331bee79ee781ff26684f8f',
 '6e87bf75a7ec7e474e90aef4188a7007']

Projects in the treeshap bottom explanations

In [20]:
list(explanations["fold0"]["treeshap"]["bottom"].keys())[:10]

['9922bc88316d50dc9fec75ace1f6428d',
 'cfbd2d9e5b4e27068f9463b029a3d777',
 '93b9f3988e3941f7acf43b2917e823fe',
 '975a0183c407402eb3fc4b32fe2df0a5',
 '77d5e0dc0c59a21425cc417f43266b99',
 'b53c6be1e99af4ad3b782cbbbc14c209',
 'd0cfa36c791f4d83123c4b9853983418',
 'bca108ff397e3f4461a6dbe966996eae',
 '1e45580a189dcba2b0a581aea5690850',
 '914ed5ecded471bcb3922ccf422d9265']

features in the lime top explanations

In [21]:
len(list(zip(*list(explanations["fold0"]["lime"]["top"].values())[0][:]))[0])

len(data_folds[0]["x_test"][data_folds[0]["x_test"]["Project ID"] == "cdc83b2eb031d999c123535bf09d24a0"].columns[1:])

575

In [22]:
print(list(zip(*list(explanations["fold0"]["lime"]["top"].values())[0][:]))[0])

('Project Cost', 'School State_New Mexico', 'Project Subject Subcategory Tree_Mathematics, Performing Arts', 'School State_Mississippi', 'School State_Nebraska', 'Project Subject Subcategory Tree_Economics, Performing Arts', 'Project Subject Subcategory Tree_College & Career Prep, Environmental Science', 'School State_Oregon', 'Project Subject Subcategory Tree_ESL, Performing Arts', 'Project Resource Category_Technology', 'Project Subject Subcategory Tree_Character Education, Social Sciences', 'Project Subject Category Tree_Health & Sports, Math & Science', 'Project Subject Subcategory Tree_Other, Social Sciences', 'School State_Missouri', 'Project Subject Subcategory Tree_Early Development, Environmental Science', 'Project Subject Subcategory Tree_Community Service, Extracurricular', 'Project Subject Subcategory Tree_Character Education, Civics & Government', 'Project Subject Subcategory Tree_Civics & Government, Community Service', 'Project Subject Subcategory Tree_Foreign Languages,

In [23]:
print(data_folds[0]["x_test"][data_folds[0]["x_test"]["Project ID"] == "cdc83b2eb031d999c123535bf09d24a0"].columns[1:])

Index(['Project Posted Month', 'Project Cost', 'School Percentage Free Lunch',
       'Teacher Project Posted Sequence', 'Teacher Success Rate',
       'Teacher Success Rate Imputed', 'School City Success Rate',
       'School City Success Rate Imputed', 'School Success Rate',
       'School Success Rate Imputed',
       ...
       'School State_Washington', 'School State_West Virginia',
       'School State_Wisconsin', 'School State_Wyoming', 'Teacher Prefix_Dr.',
       'Teacher Prefix_Mr.', 'Teacher Prefix_Mrs.', 'Teacher Prefix_Ms.',
       'Teacher Prefix_Mx.', 'Teacher Prefix_Teacher'],
      dtype='object', length=575)


features in the lime bottom explanations

In [24]:
list(zip(*list(explanations["fold0"]["lime"]["bottom"].values())[0][:10]))[0]

('School State_New Mexico',
 'Project Cost',
 'School State_Mississippi',
 'School State_Oregon',
 'Project Subject Subcategory Tree_College & Career Prep, Early Development',
 'Project Subject Category Tree_Music & The Arts, History & Civics',
 'Project Subject Subcategory Tree_Literature & Writing, Other',
 'Project Subject Subcategory Tree_Community Service, Environmental Science',
 'Project Subject Subcategory Tree_Community Service, Mathematics',
 'Project Subject Subcategory Tree_Civics & Government, Visual Arts')

features in the treeshap top explanations

In [25]:
list(zip(*list(explanations["fold0"]["treeshap"]["top"].values())[0][:10]))[0]

('Project Cost',
 'School State_Mississippi',
 'Project Resource Category_Technology',
 'Teacher Project Posted Sequence',
 'School City Success Rate',
 'School Success Rate',
 'Teacher Success Rate',
 'Teacher Success Rate Imputed',
 'School County Success Rate',
 'Project Count in State')

features in the treeshap bottom explanations

In [26]:
list(zip(*list(explanations["fold0"]["treeshap"]["bottom"].values())[0][:10]))[0]



('Project Cost',
 'School State_Illinois',
 'School Success Rate',
 'Project Resource Category_Technology',
 'Project Posted Month',
 'Teacher Project Posted Sequence',
 'Teacher Success Rate',
 'Project Count in State',
 'Project Resource Category_Books',
 'Teacher Success Rate Imputed')

In [27]:
project_explanation = explanations["fold0"]["treeshap"]["bottom"]["9922bc88316d50dc9fec75ace1f6428d"]

# Make the feature vs important score list into dictionary
project = {}
for feature in project_explanation:
    if feature[0] not in project.keys():
        project[feature[0]] = feature[1]
        
    else:
        print(f"{feature[0]} has duplicates")
        
    

In [28]:
# Function to load the pickle file and return the feature importances
def load_model_and_feat(pkl_model_path):
  # Load the model
  #model = pickle.load(open(pkl_model_path, 'rb'))
  with open(pkl_model_path,"rb") as file_handle:
    model = pickle.load(file_handle)
  # Load the feature importance array
  importance = model.feature_importances_
  # Get the feature names
  feat_names = model.feature_names_in_

  return importance, feat_names


# Function to select the top k features and plot
def select_top_features_and_plot(importance, feat_names, title):
  # Create df and sort
    df = pd.DataFrame({"Feature": feat_names, "Importance": importance})
    df_sorted = df.sort_values("Importance", ascending=False)

    # Plot
#     plt.cla()
#     plt.barh(df_sorted["Feature"], df_sorted['Importance'], label=title)
#     plt.show()

    return df_sorted

In [29]:
import glob

directory = "/storage/scratch/e17-fyp-xai/projects/e17-4yp-using-machine-learning-in-high-stake-settings/code/model_outputs/artifacts/random_forest_t_300_md_10"


pkl_files = glob.glob(f"{directory}/*.pkl")
pkl_file_names = [os.path.basename(file) for file in pkl_files]
models = sorted(pkl_file_names)

print(models)

['random_forest_t_300_md_10_fold_1_2016-01-07.pkl', 'random_forest_t_300_md_10_fold_2_2015-09-09.pkl', 'random_forest_t_300_md_10_fold_3_2015-05-12.pkl', 'random_forest_t_300_md_10_fold_4_2015-01-12.pkl', 'random_forest_t_300_md_10_fold_5_2014-09-14.pkl', 'random_forest_t_300_md_10_fold_6_2014-05-17.pkl']


In [30]:
%matplotlib inline
importance_1, feat_names_1 = load_model_and_feat(os.path.join(directory,'random_forest_t_300_md_10_fold_1_2016-01-07.pkl'))
global_explanation = select_top_features_and_plot( importance_1, feat_names_1, "Fold one")

In [31]:
global_explanation.values

array([['Project Cost', 0.2599720532503212],
       ['School Success Rate', 0.09443547578970575],
       ['School City Success Rate', 0.06234345759559309],
       ...,
       ['Project Subject Subcategory Tree_Financial Literacy, Warmth, Care & Hunger',
        0.0],
       ['Project Subject Subcategory Tree_Foreign Languages, Gym & Fitness',
        0.0],
       ['Project Subject Subcategory Tree_Extracurricular, Nutrition Education',
        0.0]], dtype=object)

### Let's say i am considering the first explanation in the global and the first explanation in the lime with compacted feature space.

The explanation pertains to the projects considered high critical according to the model's evaluation.

### Compact based on Max absolute value

In [32]:
global_explanation

,Feature,Importance
1,Project Cost,0.259972
8,School Success Rate,0.094435
6,School City Success Rate,0.062343
507,Project Resource Category_Technology,0.055903
10,School County Success Rate,0.050564
...,...,...
326,Project Subject Subcategory Tree_Financial Lit...,0.000000
328,Project Subject Subcategory Tree_Financial Lit...,0.000000
330,Project Subject Subcategory Tree_Financial Lit...,0.000000
332,Project Subject Subcategory Tree_Foreign Langu...,0.000000


In [33]:
disagreement = Disagreement(global_explanation.values, explanations["fold0"]["lime"]["top"]["cdc83b2eb031d999c123535bf09d24a0"])

In [34]:
print(disagreement.get_rank_agreement(20))

0.05


In [35]:
disagreement.sorted_explanation1[:20]

[array(['Project Cost', 0.2599720532503212], dtype=object),
 array(['School Success Rate', 0.09443547578970575], dtype=object),
 array(['School City Success Rate', 0.06234345759559309], dtype=object),
 array(['Project Resource Category_Technology', 0.05590279872535298],
       dtype=object),
 array(['School County Success Rate', 0.05056368552397339], dtype=object),
 array(['Teacher Success Rate', 0.04092739985879964], dtype=object),
 array(['Teacher Project Posted Sequence', 0.03659256423140834],
       dtype=object),
 array(['Project Posted Month', 0.036100048426852806], dtype=object),
 array(['Project Resource Category_Books', 0.0270314331657448],
       dtype=object),
 array(['Project Count in State', 0.025140087167820183], dtype=object),
 array(['School Percentage Free Lunch', 0.02389329825595476], dtype=object),
 array(['Teacher Success Rate Imputed', 0.014344914786287518], dtype=object),
 array(['School State_Illinois', 0.010733061581973777], dtype=object),
 array(['School State_

In [36]:
disagreement.sorted_explanation2[:20]

[['Project Cost', 0.05828515065993562],
 ['School State_New Mexico', -0.04561995832431883],
 ['Project Subject Subcategory Tree_Mathematics, Performing Arts',
  -0.03652875725130923],
 ['School State_Mississippi', 0.03525123751671513],
 ['School State_Nebraska', -0.030140219371630757],
 ['Project Subject Subcategory Tree_Economics, Performing Arts',
  0.028696205307718412],
 ['Project Subject Subcategory Tree_College & Career Prep, Environmental Science',
  0.025975377818970764],
 ['School State_Oregon', 0.02477959436814468],
 ['Project Subject Subcategory Tree_ESL, Performing Arts',
  0.024418609316633387],
 ['Project Resource Category_Technology', 0.02412976165593627],
 ['Project Subject Subcategory Tree_Character Education, Social Sciences',
  -0.023669453164068096],
 ['Project Subject Category Tree_Health & Sports, Math & Science',
  -0.023382474876373693],
 ['Project Subject Subcategory Tree_Other, Social Sciences',
  0.023007167202545663],
 ['School State_Missouri', -0.0227186955

In [37]:
max_compactor = MaxCompactor(config.CATEGORICAL_COLS)

In [38]:
disagreement.compact_features(max_compactor)

In [39]:
disagreement.sorted_explanation1

[['Project Cost', 0.2599720532503212],
 ['School Success Rate', 0.09443547578970575],
 ['School City Success Rate', 0.06234345759559309],
 ['Project Resource Category', 0.05590279872535298],
 ['School County Success Rate', 0.05056368552397339],
 ['Teacher Success Rate', 0.04092739985879964],
 ['Teacher Project Posted Sequence', 0.03659256423140834],
 ['Project Posted Month', 0.036100048426852806],
 ['Project Count in State', 0.025140087167820183],
 ['School Percentage Free Lunch', 0.02389329825595476],
 ['Teacher Success Rate Imputed', 0.014344914786287518],
 ['School State', 0.010733061581973777],
 ['School Success Rate Imputed', 0.006876595344258723],
 ['Teacher Prefix', 0.005092031353488309],
 ['Project Type', 0.0047677704807176135],
 ['School Metro Type', 0.003943482754726902],
 ['Project Grade Level Category', 0.0029688577721777134],
 ['School City Success Rate Imputed', 0.0022854201305964046],
 ['Project Subject Category Tree', 0.0021397892482132932],
 ['Project Subject Subcatego

In [40]:
disagreement.get_disagreement(20, None)

{'feature_agreement': 0.95,
 'rank_agreement': 0.25,
 'sign_agreement': 0.6,
 'signed_rank_agreement': 0.2}

In [41]:
disagreement.sorted_explanation1[:20]


[['Project Cost', 0.2599720532503212],
 ['School Success Rate', 0.09443547578970575],
 ['School City Success Rate', 0.06234345759559309],
 ['Project Resource Category', 0.05590279872535298],
 ['School County Success Rate', 0.05056368552397339],
 ['Teacher Success Rate', 0.04092739985879964],
 ['Teacher Project Posted Sequence', 0.03659256423140834],
 ['Project Posted Month', 0.036100048426852806],
 ['Project Count in State', 0.025140087167820183],
 ['School Percentage Free Lunch', 0.02389329825595476],
 ['Teacher Success Rate Imputed', 0.014344914786287518],
 ['School State', 0.010733061581973777],
 ['School Success Rate Imputed', 0.006876595344258723],
 ['Teacher Prefix', 0.005092031353488309],
 ['Project Type', 0.0047677704807176135],
 ['School Metro Type', 0.003943482754726902],
 ['Project Grade Level Category', 0.0029688577721777134],
 ['School City Success Rate Imputed', 0.0022854201305964046],
 ['Project Subject Category Tree', 0.0021397892482132932],
 ['Project Subject Subcatego

In [42]:
disagreement.sorted_explanation2[:20]


[['Project Cost', 0.05828515065993562],
 ['School State', -0.04561995832431883],
 ['Project Subject Subcategory Tree', -0.03652875725130923],
 ['Project Resource Category', 0.02412976165593627],
 ['Project Subject Category Tree', -0.023382474876373693],
 ['Project Type', -0.01874909636736988],
 ['Teacher Project Posted Sequence', 0.01764274655322783],
 ['School City Success Rate', 0.01762791127218422],
 ['School Success Rate', 0.01439123893716053],
 ['School County Success Rate', 0.010396124467336478],
 ['Teacher Prefix', -0.009602694849670359],
 ['Teacher Success Rate Imputed', 0.008948085524911165],
 ['Teacher Success Rate', 0.00887785459302942],
 ['School County Success Rate Imputed', -0.0086799413234964],
 ['Project Posted Month', 0.005683481725688235],
 ['School Metro Type', 0.00521351220862078],
 ['School Success Rate Imputed', -0.005064600567585159],
 ['School City Success Rate Imputed', -0.004505058261636036],
 ['School Percentage Free Lunch', 0.0038863764057131076],
 ['Project

### Compact based on Activated value

In [43]:
disagreement = Disagreement(global_explanation.values, explanations["fold0"]["lime"]["top"]["cdc83b2eb031d999c123535bf09d24a0"])

In [44]:
disagreement.sorted_explanation1[:20]

[array(['Project Cost', 0.2599720532503212], dtype=object),
 array(['School Success Rate', 0.09443547578970575], dtype=object),
 array(['School City Success Rate', 0.06234345759559309], dtype=object),
 array(['Project Resource Category_Technology', 0.05590279872535298],
       dtype=object),
 array(['School County Success Rate', 0.05056368552397339], dtype=object),
 array(['Teacher Success Rate', 0.04092739985879964], dtype=object),
 array(['Teacher Project Posted Sequence', 0.03659256423140834],
       dtype=object),
 array(['Project Posted Month', 0.036100048426852806], dtype=object),
 array(['Project Resource Category_Books', 0.0270314331657448],
       dtype=object),
 array(['Project Count in State', 0.025140087167820183], dtype=object),
 array(['School Percentage Free Lunch', 0.02389329825595476], dtype=object),
 array(['Teacher Success Rate Imputed', 0.014344914786287518], dtype=object),
 array(['School State_Illinois', 0.010733061581973777], dtype=object),
 array(['School State_

In [45]:
disagreement.sorted_explanation2[:20]

[['Project Cost', 0.05828515065993562],
 ['School State_New Mexico', -0.04561995832431883],
 ['Project Subject Subcategory Tree_Mathematics, Performing Arts',
  -0.03652875725130923],
 ['School State_Mississippi', 0.03525123751671513],
 ['School State_Nebraska', -0.030140219371630757],
 ['Project Subject Subcategory Tree_Economics, Performing Arts',
  0.028696205307718412],
 ['Project Subject Subcategory Tree_College & Career Prep, Environmental Science',
  0.025975377818970764],
 ['School State_Oregon', 0.02477959436814468],
 ['Project Subject Subcategory Tree_ESL, Performing Arts',
  0.024418609316633387],
 ['Project Resource Category_Technology', 0.02412976165593627],
 ['Project Subject Subcategory Tree_Character Education, Social Sciences',
  -0.023669453164068096],
 ['Project Subject Category Tree_Health & Sports, Math & Science',
  -0.023382474876373693],
 ['Project Subject Subcategory Tree_Other, Social Sciences',
  0.023007167202545663],
 ['School State_Missouri', -0.0227186955

In [46]:
activated_features = {}
for feature in config.CATEGORICAL_COLS:
    activated_features[feature] = f"{feature}_{raw_data[raw_data['Project ID'] == 'cdc83b2eb031d999c123535bf09d24a0' ][feature].values[0]}"

In [47]:
activated_compactor = ActivatedCompactor(activated_features)

In [48]:
disagreement.compact_features(activated_compactor)

In [49]:
disagreement.get_disagreement(20, None)

{'feature_agreement': 0.95,
 'rank_agreement': 0.15,
 'sign_agreement': 0.75,
 'signed_rank_agreement': 0.1}

In [50]:
disagreement.sorted_explanation1[:20]


[['Project Cost', 0.2599720532503212],
 ['School Success Rate', 0.09443547578970575],
 ['School City Success Rate', 0.06234345759559309],
 ['Project Resource Category', 0.05590279872535298],
 ['School County Success Rate', 0.05056368552397339],
 ['Teacher Success Rate', 0.04092739985879964],
 ['Teacher Project Posted Sequence', 0.03659256423140834],
 ['Project Posted Month', 0.036100048426852806],
 ['Project Count in State', 0.025140087167820183],
 ['School Percentage Free Lunch', 0.02389329825595476],
 ['Teacher Success Rate Imputed', 0.014344914786287518],
 ['School State', 0.010496705563170072],
 ['School Success Rate Imputed', 0.006876595344258723],
 ['School Metro Type', 0.003903959116131502],
 ['Project Grade Level Category', 0.0029688577721777134],
 ['Teacher Prefix', 0.00267333382971531],
 ['School City Success Rate Imputed', 0.0022854201305964046],
 ['Project Subject Category Tree', 0.0021397892482132932],
 ['Project Type', 0.0016263807739691878],
 ['School County Success Rate

In [51]:
disagreement.sorted_explanation2[:20]


[['Project Cost', 0.05828515065993562],
 ['School State', 0.03525123751671513],
 ['Project Resource Category', 0.02412976165593627],
 ['Teacher Project Posted Sequence', 0.01764274655322783],
 ['School City Success Rate', 0.01762791127218422],
 ['School Success Rate', 0.01439123893716053],
 ['School County Success Rate', 0.010396124467336478],
 ['Teacher Success Rate Imputed', 0.008948085524911165],
 ['Teacher Success Rate', 0.00887785459302942],
 ['School County Success Rate Imputed', -0.0086799413234964],
 ['Project Posted Month', 0.005683481725688235],
 ['School Metro Type', 0.00521351220862078],
 ['School Success Rate Imputed', -0.005064600567585159],
 ['School City Success Rate Imputed', -0.004505058261636036],
 ['School Percentage Free Lunch', 0.0038863764057131076],
 ['Project Count in State', 0.003006706209883576],
 ['Project Grade Level Category', -0.0017520537476009201],
 ['Project Subject Category Tree', 0.0017236348728642999],
 ['Project Subject Subcategory Tree', 1.5403463

### Compact based on Not Activated value

In [52]:
from compactor.MaxNonActivatedCompactor import MaxNonActivatedCompactor

In [53]:
disagreement = Disagreement(global_explanation.values, explanations["fold0"]["lime"]["top"]["cdc83b2eb031d999c123535bf09d24a0"])

In [54]:
disagreement.sorted_explanation1[:20]

[array(['Project Cost', 0.2599720532503212], dtype=object),
 array(['School Success Rate', 0.09443547578970575], dtype=object),
 array(['School City Success Rate', 0.06234345759559309], dtype=object),
 array(['Project Resource Category_Technology', 0.05590279872535298],
       dtype=object),
 array(['School County Success Rate', 0.05056368552397339], dtype=object),
 array(['Teacher Success Rate', 0.04092739985879964], dtype=object),
 array(['Teacher Project Posted Sequence', 0.03659256423140834],
       dtype=object),
 array(['Project Posted Month', 0.036100048426852806], dtype=object),
 array(['Project Resource Category_Books', 0.0270314331657448],
       dtype=object),
 array(['Project Count in State', 0.025140087167820183], dtype=object),
 array(['School Percentage Free Lunch', 0.02389329825595476], dtype=object),
 array(['Teacher Success Rate Imputed', 0.014344914786287518], dtype=object),
 array(['School State_Illinois', 0.010733061581973777], dtype=object),
 array(['School State_

In [55]:
disagreement.sorted_explanation2[:20]

[['Project Cost', 0.05828515065993562],
 ['School State_New Mexico', -0.04561995832431883],
 ['Project Subject Subcategory Tree_Mathematics, Performing Arts',
  -0.03652875725130923],
 ['School State_Mississippi', 0.03525123751671513],
 ['School State_Nebraska', -0.030140219371630757],
 ['Project Subject Subcategory Tree_Economics, Performing Arts',
  0.028696205307718412],
 ['Project Subject Subcategory Tree_College & Career Prep, Environmental Science',
  0.025975377818970764],
 ['School State_Oregon', 0.02477959436814468],
 ['Project Subject Subcategory Tree_ESL, Performing Arts',
  0.024418609316633387],
 ['Project Resource Category_Technology', 0.02412976165593627],
 ['Project Subject Subcategory Tree_Character Education, Social Sciences',
  -0.023669453164068096],
 ['Project Subject Category Tree_Health & Sports, Math & Science',
  -0.023382474876373693],
 ['Project Subject Subcategory Tree_Other, Social Sciences',
  0.023007167202545663],
 ['School State_Missouri', -0.0227186955

In [56]:
activated_features = {}
for feature in config.CATEGORICAL_COLS:
    activated_features[feature] = f"{feature}_{raw_data[raw_data['Project ID'] == 'cdc83b2eb031d999c123535bf09d24a0' ][feature].values[0]}"

In [57]:
max_activated_compactor = MaxNonActivatedCompactor(activated_features)

In [58]:
disagreement.compact_features(max_activated_compactor)

In [59]:
disagreement.get_disagreement(20, None)

{'feature_agreement': 0.95,
 'rank_agreement': 0.05,
 'sign_agreement': 0.55,
 'signed_rank_agreement': 0.05}

In [60]:
disagreement.sorted_explanation1[:20]


[['Project Cost', 0.2599720532503212],
 ['School Success Rate', 0.09443547578970575],
 ['School City Success Rate', 0.06234345759559309],
 ['School County Success Rate', 0.05056368552397339],
 ['Teacher Success Rate', 0.04092739985879964],
 ['Teacher Project Posted Sequence', 0.03659256423140834],
 ['Project Posted Month', 0.036100048426852806],
 ['Project Resource Category', 0.0270314331657448],
 ['Project Count in State', 0.025140087167820183],
 ['School Percentage Free Lunch', 0.02389329825595476],
 ['Teacher Success Rate Imputed', 0.014344914786287518],
 ['School State', 0.010733061581973777],
 ['School Success Rate Imputed', 0.006876595344258723],
 ['Teacher Prefix', 0.005092031353488309],
 ['Project Type', 0.0047677704807176135],
 ['School Metro Type', 0.003943482754726902],
 ['Project Grade Level Category', 0.0024169498930726044],
 ['School City Success Rate Imputed', 0.0022854201305964046],
 ['Project Subject Category Tree', 0.0019586068659635967],
 ['Project Subject Subcategor

In [61]:
disagreement.sorted_explanation2[:20]


[['Project Cost', 0.05828515065993562],
 ['School State', -0.04561995832431883],
 ['Project Subject Subcategory Tree', -0.03652875725130923],
 ['Project Subject Category Tree', -0.023382474876373693],
 ['Project Resource Category', 0.02017981543707445],
 ['Project Type', -0.01874909636736988],
 ['Teacher Project Posted Sequence', 0.01764274655322783],
 ['School City Success Rate', 0.01762791127218422],
 ['School Success Rate', 0.01439123893716053],
 ['School County Success Rate', 0.010396124467336478],
 ['Teacher Prefix', -0.009602694849670359],
 ['Teacher Success Rate Imputed', 0.008948085524911165],
 ['Teacher Success Rate', 0.00887785459302942],
 ['School County Success Rate Imputed', -0.0086799413234964],
 ['Project Posted Month', 0.005683481725688235],
 ['School Success Rate Imputed', -0.005064600567585159],
 ['School City Success Rate Imputed', -0.004505058261636036],
 ['School Percentage Free Lunch', 0.0038863764057131076],
 ['Project Count in State', 0.003006706209883576],
 ['S

#### All the metrics shows that there is significant disagreement for the first explanation

Based on top 20 features in the first explanation of lime and treeshap


Feature Agreement 0.15: The feature agreement score is 0.15. This indicates that the top 20 features in Lime's explanation are different from TreeSHAP's top 20 features.

Rank Agreement 0.05: The rank agreement score is 0.05. This means that not only are the top 20 features different, but there is also a significant difference in the order of these features between Lime and TreeSHAP.

Sign Agreement 0.15: The sign agreement score is 0.15. This suggests that the top 20 features in Lime's explanation have different features compared to TreeSHAP's top 20 features when considering the sign of the feature values.

Signed Rank Agreement 0.05: The signed rank agreement score is 0.05. This implies that the top 20 features in Lime's explanation not only differ in terms of the features themselves but also in their rank and sign when compared to TreeSHAP's top 20 features.

### Let's say i am considering the last explanation in the treeshap and the last explanation in the lime with compacted feature space.

The explanation pertains to the projects considered less critical according to the model's evaluation.

### Compact based on Max absolute value

In [62]:
disagreement = Disagreement(global_explanation.values, explanations["fold0"]["lime"]["bottom"]["914ed5ecded471bcb3922ccf422d9265"])

In [63]:
disagreement.sorted_explanation1[:20]

[array(['Project Cost', 0.2599720532503212], dtype=object),
 array(['School Success Rate', 0.09443547578970575], dtype=object),
 array(['School City Success Rate', 0.06234345759559309], dtype=object),
 array(['Project Resource Category_Technology', 0.05590279872535298],
       dtype=object),
 array(['School County Success Rate', 0.05056368552397339], dtype=object),
 array(['Teacher Success Rate', 0.04092739985879964], dtype=object),
 array(['Teacher Project Posted Sequence', 0.03659256423140834],
       dtype=object),
 array(['Project Posted Month', 0.036100048426852806], dtype=object),
 array(['Project Resource Category_Books', 0.0270314331657448],
       dtype=object),
 array(['Project Count in State', 0.025140087167820183], dtype=object),
 array(['School Percentage Free Lunch', 0.02389329825595476], dtype=object),
 array(['Teacher Success Rate Imputed', 0.014344914786287518], dtype=object),
 array(['School State_Illinois', 0.010733061581973777], dtype=object),
 array(['School State_

In [64]:
disagreement.sorted_explanation2[:20]

[['School State_New Mexico', -0.039534328867710765],
 ['Project Subject Subcategory Tree_Early Development, Team Sports',
  0.03927456784041016],
 ['School State_Oregon', 0.0359405748502581],
 ['Project Subject Category Tree_Health & Sports, History & Civics',
  0.03567932952839871],
 ['Project Subject Subcategory Tree_Character Education, Mathematics',
  0.031610236047156556],
 ['Project Cost', -0.02952929601923634],
 ['Project Subject Subcategory Tree_Foreign Languages, Literature & Writing',
  -0.029357881597820344],
 ['Project Subject Subcategory Tree_Environmental Science, Music',
  0.0287163292291289],
 ['Project Subject Subcategory Tree_College & Career Prep, Financial Literacy',
  0.028657001397669433],
 ['Project Subject Subcategory Tree_Social Sciences, Visual Arts',
  0.028511500035182127],
 ['Project Subject Subcategory Tree_Economics, Financial Literacy',
  -0.0282102779224615],
 ['Project Subject Subcategory Tree_ESL, History & Geography',
  0.02800016263929736],
 ['Proje

In [65]:
max_compactor = MaxCompactor(config.CATEGORICAL_COLS)

In [66]:
disagreement.compact_features(max_compactor)

In [67]:
disagreement.sorted_explanation1

[['Project Cost', 0.2599720532503212],
 ['School Success Rate', 0.09443547578970575],
 ['School City Success Rate', 0.06234345759559309],
 ['Project Resource Category', 0.05590279872535298],
 ['School County Success Rate', 0.05056368552397339],
 ['Teacher Success Rate', 0.04092739985879964],
 ['Teacher Project Posted Sequence', 0.03659256423140834],
 ['Project Posted Month', 0.036100048426852806],
 ['Project Count in State', 0.025140087167820183],
 ['School Percentage Free Lunch', 0.02389329825595476],
 ['Teacher Success Rate Imputed', 0.014344914786287518],
 ['School State', 0.010733061581973777],
 ['School Success Rate Imputed', 0.006876595344258723],
 ['Teacher Prefix', 0.005092031353488309],
 ['Project Type', 0.0047677704807176135],
 ['School Metro Type', 0.003943482754726902],
 ['Project Grade Level Category', 0.0029688577721777134],
 ['School City Success Rate Imputed', 0.0022854201305964046],
 ['Project Subject Category Tree', 0.0021397892482132932],
 ['Project Subject Subcatego

In [68]:
disagreement.get_disagreement(20, None)

{'feature_agreement': 0.95,
 'rank_agreement': 0.0,
 'sign_agreement': 0.2,
 'signed_rank_agreement': 0.0}

In [69]:
disagreement.sorted_explanation1[:20]


[['Project Cost', 0.2599720532503212],
 ['School Success Rate', 0.09443547578970575],
 ['School City Success Rate', 0.06234345759559309],
 ['Project Resource Category', 0.05590279872535298],
 ['School County Success Rate', 0.05056368552397339],
 ['Teacher Success Rate', 0.04092739985879964],
 ['Teacher Project Posted Sequence', 0.03659256423140834],
 ['Project Posted Month', 0.036100048426852806],
 ['Project Count in State', 0.025140087167820183],
 ['School Percentage Free Lunch', 0.02389329825595476],
 ['Teacher Success Rate Imputed', 0.014344914786287518],
 ['School State', 0.010733061581973777],
 ['School Success Rate Imputed', 0.006876595344258723],
 ['Teacher Prefix', 0.005092031353488309],
 ['Project Type', 0.0047677704807176135],
 ['School Metro Type', 0.003943482754726902],
 ['Project Grade Level Category', 0.0029688577721777134],
 ['School City Success Rate Imputed', 0.0022854201305964046],
 ['Project Subject Category Tree', 0.0021397892482132932],
 ['Project Subject Subcatego

In [70]:
disagreement.sorted_explanation2[:20]


[['School State', -0.039534328867710765],
 ['Project Subject Subcategory Tree', 0.03927456784041016],
 ['Project Subject Category Tree', 0.03567932952839871],
 ['Project Cost', -0.02952929601923634],
 ['School Success Rate', -0.025776909203259216],
 ['Project Resource Category', -0.023836287905638617],
 ['Project Type', -0.01503048269510653],
 ['Teacher Success Rate', 0.012813475583557795],
 ['Teacher Prefix', -0.009125173179515167],
 ['Teacher Success Rate Imputed', -0.009009803255737941],
 ['School Percentage Free Lunch', -0.008126922865584042],
 ['School City Success Rate', -0.007641181462543814],
 ['Teacher Project Posted Sequence', -0.0068610916964242825],
 ['Project Posted Month', 0.006653825060046289],
 ['School County Success Rate Imputed', -0.006001961298228468],
 ['School City Success Rate Imputed', -0.005584178358922777],
 ['School Metro Type', -0.0049755677668917855],
 ['School Success Rate Imputed', -0.004736606280679097],
 ['School County Success Rate', -0.004302345526877

### Compact based on Activated value

In [71]:
disagreement = Disagreement(global_explanation.values, explanations["fold0"]["lime"]["bottom"]["914ed5ecded471bcb3922ccf422d9265"])

In [72]:
disagreement.sorted_explanation1[:20]

[array(['Project Cost', 0.2599720532503212], dtype=object),
 array(['School Success Rate', 0.09443547578970575], dtype=object),
 array(['School City Success Rate', 0.06234345759559309], dtype=object),
 array(['Project Resource Category_Technology', 0.05590279872535298],
       dtype=object),
 array(['School County Success Rate', 0.05056368552397339], dtype=object),
 array(['Teacher Success Rate', 0.04092739985879964], dtype=object),
 array(['Teacher Project Posted Sequence', 0.03659256423140834],
       dtype=object),
 array(['Project Posted Month', 0.036100048426852806], dtype=object),
 array(['Project Resource Category_Books', 0.0270314331657448],
       dtype=object),
 array(['Project Count in State', 0.025140087167820183], dtype=object),
 array(['School Percentage Free Lunch', 0.02389329825595476], dtype=object),
 array(['Teacher Success Rate Imputed', 0.014344914786287518], dtype=object),
 array(['School State_Illinois', 0.010733061581973777], dtype=object),
 array(['School State_

In [73]:
disagreement.sorted_explanation2[:20]

[['School State_New Mexico', -0.039534328867710765],
 ['Project Subject Subcategory Tree_Early Development, Team Sports',
  0.03927456784041016],
 ['School State_Oregon', 0.0359405748502581],
 ['Project Subject Category Tree_Health & Sports, History & Civics',
  0.03567932952839871],
 ['Project Subject Subcategory Tree_Character Education, Mathematics',
  0.031610236047156556],
 ['Project Cost', -0.02952929601923634],
 ['Project Subject Subcategory Tree_Foreign Languages, Literature & Writing',
  -0.029357881597820344],
 ['Project Subject Subcategory Tree_Environmental Science, Music',
  0.0287163292291289],
 ['Project Subject Subcategory Tree_College & Career Prep, Financial Literacy',
  0.028657001397669433],
 ['Project Subject Subcategory Tree_Social Sciences, Visual Arts',
  0.028511500035182127],
 ['Project Subject Subcategory Tree_Economics, Financial Literacy',
  -0.0282102779224615],
 ['Project Subject Subcategory Tree_ESL, History & Geography',
  0.02800016263929736],
 ['Proje

In [74]:
activated_features = {}
for feature in config.CATEGORICAL_COLS:
    activated_features[feature] = f"{feature}_{raw_data[raw_data['Project ID'] == '914ed5ecded471bcb3922ccf422d9265' ][feature].values[0]}"

In [75]:
activated_compactor = ActivatedCompactor(activated_features)

In [76]:
disagreement.compact_features(activated_compactor)

In [77]:
disagreement.get_disagreement(20, None)

{'feature_agreement': 0.95,
 'rank_agreement': 0.15,
 'sign_agreement': 0.3,
 'signed_rank_agreement': 0.0}

In [78]:
disagreement.sorted_explanation1[:20]


[['Project Cost', 0.2599720532503212],
 ['School Success Rate', 0.09443547578970575],
 ['School City Success Rate', 0.06234345759559309],
 ['School County Success Rate', 0.05056368552397339],
 ['Teacher Success Rate', 0.04092739985879964],
 ['Teacher Project Posted Sequence', 0.03659256423140834],
 ['Project Posted Month', 0.036100048426852806],
 ['Project Resource Category', 0.0270314331657448],
 ['Project Count in State', 0.025140087167820183],
 ['School Percentage Free Lunch', 0.02389329825595476],
 ['Teacher Success Rate Imputed', 0.014344914786287518],
 ['School Success Rate Imputed', 0.006876595344258723],
 ['School Metro Type', 0.003943482754726902],
 ['Teacher Prefix', 0.00267333382971531],
 ['School City Success Rate Imputed', 0.0022854201305964046],
 ['Project Subject Category Tree', 0.0019289178869664931],
 ['Project Grade Level Category', 0.0018638039483081996],
 ['Project Type', 0.0016263807739691878],
 ['Project Subject Subcategory Tree', 0.0015565784693995046],
 ['School

In [79]:
disagreement.sorted_explanation2[:20]


[['Project Cost', -0.02952929601923634],
 ['School Success Rate', -0.025776909203259216],
 ['Project Resource Category', -0.019918174049804022],
 ['Teacher Success Rate', 0.012813475583557795],
 ['Teacher Success Rate Imputed', -0.009009803255737941],
 ['School Percentage Free Lunch', -0.008126922865584042],
 ['School City Success Rate', -0.007641181462543814],
 ['Teacher Project Posted Sequence', -0.0068610916964242825],
 ['Project Posted Month', 0.006653825060046289],
 ['School County Success Rate Imputed', -0.006001961298228468],
 ['School City Success Rate Imputed', -0.005584178358922777],
 ['School Success Rate Imputed', -0.004736606280679097],
 ['School County Success Rate', -0.004302345526877994],
 ['Project Subject Category Tree', 0.002089182444430119],
 ['Project Count in State', 0.002015427891416808],
 ['School State', 0.0009187866139342211],
 ['School Metro Type', -0.0008567512498406881],
 ['Teacher Prefix', 0.0006970026340453418],
 ['Project Grade Level Category', 0.0006967

### Compact based on Not Activated value

In [80]:
from compactor.MaxNonActivatedCompactor import MaxNonActivatedCompactor

In [81]:
disagreement = Disagreement(global_explanation.values, explanations["fold0"]["lime"]["bottom"]["914ed5ecded471bcb3922ccf422d9265"])

In [82]:
disagreement.sorted_explanation1[:20]

[array(['Project Cost', 0.2599720532503212], dtype=object),
 array(['School Success Rate', 0.09443547578970575], dtype=object),
 array(['School City Success Rate', 0.06234345759559309], dtype=object),
 array(['Project Resource Category_Technology', 0.05590279872535298],
       dtype=object),
 array(['School County Success Rate', 0.05056368552397339], dtype=object),
 array(['Teacher Success Rate', 0.04092739985879964], dtype=object),
 array(['Teacher Project Posted Sequence', 0.03659256423140834],
       dtype=object),
 array(['Project Posted Month', 0.036100048426852806], dtype=object),
 array(['Project Resource Category_Books', 0.0270314331657448],
       dtype=object),
 array(['Project Count in State', 0.025140087167820183], dtype=object),
 array(['School Percentage Free Lunch', 0.02389329825595476], dtype=object),
 array(['Teacher Success Rate Imputed', 0.014344914786287518], dtype=object),
 array(['School State_Illinois', 0.010733061581973777], dtype=object),
 array(['School State_

In [83]:
disagreement.sorted_explanation2[:20]

[['School State_New Mexico', -0.039534328867710765],
 ['Project Subject Subcategory Tree_Early Development, Team Sports',
  0.03927456784041016],
 ['School State_Oregon', 0.0359405748502581],
 ['Project Subject Category Tree_Health & Sports, History & Civics',
  0.03567932952839871],
 ['Project Subject Subcategory Tree_Character Education, Mathematics',
  0.031610236047156556],
 ['Project Cost', -0.02952929601923634],
 ['Project Subject Subcategory Tree_Foreign Languages, Literature & Writing',
  -0.029357881597820344],
 ['Project Subject Subcategory Tree_Environmental Science, Music',
  0.0287163292291289],
 ['Project Subject Subcategory Tree_College & Career Prep, Financial Literacy',
  0.028657001397669433],
 ['Project Subject Subcategory Tree_Social Sciences, Visual Arts',
  0.028511500035182127],
 ['Project Subject Subcategory Tree_Economics, Financial Literacy',
  -0.0282102779224615],
 ['Project Subject Subcategory Tree_ESL, History & Geography',
  0.02800016263929736],
 ['Proje

In [84]:
activated_features = {}
for feature in config.CATEGORICAL_COLS:
    activated_features[feature] = f"{feature}_{raw_data[raw_data['Project ID'] == '914ed5ecded471bcb3922ccf422d9265' ][feature].values[0]}"

In [85]:
max_activated_compactor = MaxNonActivatedCompactor(activated_features)

In [86]:
disagreement.compact_features(max_activated_compactor)

In [87]:
disagreement.get_disagreement(20, None)

{'feature_agreement': 0.95,
 'rank_agreement': 0.0,
 'sign_agreement': 0.15,
 'signed_rank_agreement': 0.0}

In [88]:
disagreement.sorted_explanation1[:20]


[['Project Cost', 0.2599720532503212],
 ['School Success Rate', 0.09443547578970575],
 ['School City Success Rate', 0.06234345759559309],
 ['Project Resource Category', 0.05590279872535298],
 ['School County Success Rate', 0.05056368552397339],
 ['Teacher Success Rate', 0.04092739985879964],
 ['Teacher Project Posted Sequence', 0.03659256423140834],
 ['Project Posted Month', 0.036100048426852806],
 ['Project Count in State', 0.025140087167820183],
 ['School Percentage Free Lunch', 0.02389329825595476],
 ['Teacher Success Rate Imputed', 0.014344914786287518],
 ['School State', 0.010733061581973777],
 ['School Success Rate Imputed', 0.006876595344258723],
 ['Teacher Prefix', 0.005092031353488309],
 ['Project Type', 0.0047677704807176135],
 ['School Metro Type', 0.003903959116131502],
 ['Project Grade Level Category', 0.0029688577721777134],
 ['School City Success Rate Imputed', 0.0022854201305964046],
 ['Project Subject Category Tree', 0.0021397892482132932],
 ['School County Success Rat

In [89]:
disagreement.sorted_explanation2[:20]


[['School State', -0.039534328867710765],
 ['Project Subject Subcategory Tree', 0.03927456784041016],
 ['Project Subject Category Tree', 0.03567932952839871],
 ['Project Cost', -0.02952929601923634],
 ['School Success Rate', -0.025776909203259216],
 ['Project Resource Category', -0.023836287905638617],
 ['Project Type', -0.01503048269510653],
 ['Teacher Success Rate', 0.012813475583557795],
 ['Teacher Prefix', -0.009125173179515167],
 ['Teacher Success Rate Imputed', -0.009009803255737941],
 ['School Percentage Free Lunch', -0.008126922865584042],
 ['School City Success Rate', -0.007641181462543814],
 ['Teacher Project Posted Sequence', -0.0068610916964242825],
 ['Project Posted Month', 0.006653825060046289],
 ['School County Success Rate Imputed', -0.006001961298228468],
 ['School City Success Rate Imputed', -0.005584178358922777],
 ['School Metro Type', -0.0049755677668917855],
 ['School Success Rate Imputed', -0.004736606280679097],
 ['School County Success Rate', -0.004302345526877

#### All the metrics shows that there is significant disagreement for the first explanation

Based on top 20 features in the first explanation of lime and treeshap


Feature Agreement 0.15: The feature agreement score is 0.15. This indicates that the top 20 features in Lime's explanation are different from TreeSHAP's top 20 features.

Rank Agreement 0.05: The rank agreement score is 0.05. This means that not only are the top 20 features different, but there is also a significant difference in the order of these features between Lime and TreeSHAP.

Sign Agreement 0.15: The sign agreement score is 0.15. This suggests that the top 20 features in Lime's explanation have different features compared to TreeSHAP's top 20 features when considering the sign of the feature values.

Signed Rank Agreement 0.05: The signed rank agreement score is 0.05. This implies that the top 20 features in Lime's explanation not only differ in terms of the features themselves but also in their rank and sign when compared to TreeSHAP's top 20 features.

In [90]:
from compactor.MaxNonActivatedCompactor import MaxNonActivatedCompactor
from compactor.MaxCompactor import MaxCompactor
from compactor.ActivatedCompactor import ActivatedCompactor

## Function to calculate average Disgreement

In [120]:
def global_disagreement_average(global_explanations: dict , explanations2: dict , k: int, features_F: list , method = None, raw_data = None, debug = False) -> dict:
    disagreement_mean = {'feature_agreement': 0.0,
     'rank_agreement': 0.0,
     'sign_agreement': 0.0,
     'signed_rank_agreement': 0.0}
    
    for project_id in explanations2.keys():
        if debug:
            print(f"******************Project ID: {project_id}************************")
        
        if len(explanations2[project_id]) < k:
            raise ValueError(f"K should be less than {len(explanations2[project_id])}")
        
        disagreement_calc_2 = Disagreement(global_explanations , explanations2[project_id])
        
        if debug:
            print("Initial Explanation 1 \n",disagreement_calc_2.sorted_explanation1[:k])
            print("Initial Explanation 2 \n",disagreement_calc_2.sorted_explanation2[:k])
            
        if method=="max":
            max_compactor = MaxCompactor(config.CATEGORICAL_COLS)
            disagreement_calc_2.compact_features(max_compactor)
            
        elif method == "activated" and raw_data is not None:
            activated_features = {}
            for feature in config.CATEGORICAL_COLS:
                activated_features[feature] = f"{feature}_{raw_data[raw_data['Project ID'] == project_id ][feature].values[0]}"
            activated_compactor = ActivatedCompactor(activated_features)
            disagreement_calc_2.compact_features(activated_compactor)
            
        elif method == "maxnonactivated" and raw_data is not None:
            activated_features = {}
            for feature in config.CATEGORICAL_COLS:
                activated_features[feature] = f"{feature}_{raw_data[raw_data['Project ID'] == project_id ][feature].values[0]}"
            max_non_activated_compactor = MaxNonActivatedCompactor(activated_features)
            disagreement_calc_2.compact_features(max_non_activated_compactor)
            
        if debug:
            print("After Compact Explanation 1 \n",disagreement_calc_2.sorted_explanation1[:k])
            print("After Compact Explanation 2 \n",disagreement_calc_2.sorted_explanation2[:k])
            
        disagreement = disagreement_calc_2.get_disagreement(k, features_F)
        
        if debug:
            print("Disagreement : ", disagreement)
        
        for key in list(disagreement_mean.keys()):
            if disagreement[key] == None:
                raise ValueError(f"The value for key '{key}' in disagreement_mean is None.")
            disagreement_mean[key] += disagreement[key]
        
        if debug:
            print("****************************************")

    for key in list(disagreement_mean.keys()):
        disagreement_mean[key] = round(disagreement_mean[key] / len(explanations2.keys()),3)
    
    if debug:
        print("*********************************************")
        print(disagreement_mean)
        print("*********************************************")

    
    return disagreement_mean

In [141]:

def disgreement_mean_across_top_bottom_500(global_explanations, explanations2, k_range, method=None, raw_data=None, debug=False):
    disgreement_mean_across__top_500_various_k = {'feature_agreement': [],
         'rank_agreement': [],
         'sign_agreement': [],
         'signed_rank_agreement': []}


    
        
    for i in range(1,k_range,1):
        
        explanations = global_disagreement_average(global_explanations, explanations2["top"], i, None, method, raw_data,  debug)

        for key in list(disgreement_mean_across__top_500_various_k.keys()):
            disgreement_mean_across__top_500_various_k[key].append(explanations[key])



    disgreement_mean_across__bottom_500_various_k = {'feature_agreement': [],
         'rank_agreement': [],
         'sign_agreement': [],
         'signed_rank_agreement': []}

    for i in range(1,k_range,1):
        explanations = global_disagreement_average(global_explanations, explanations2["bottom"], i, None,method, raw_data, debug)

        for key in list(disgreement_mean_across__bottom_500_various_k.keys()):
            disgreement_mean_across__bottom_500_various_k[key].append(explanations[key])
            
    return disgreement_mean_across__top_500_various_k, disgreement_mean_across__bottom_500_various_k


    

In [142]:

def disgreement_mean_at_k(explanations, k, method=None, raw_data=None, debug=True):
    

    disgreement_mean_across__top_500_at_k = disagreement_average(explanations["fold0"]["lime"]["top"], explanations["fold0"]["treeshap"]["top"], k, None,method, raw_data, debug)

    disgreement_mean_across__bottom_500_various_k = disagreement_average(explanations["fold0"]["lime"]["bottom"], explanations["fold0"]["treeshap"]["bottom"], k, None,method, raw_data, debug)

    return disgreement_mean_across__top_500_at_k, disgreement_mean_across__bottom_500_various_k


    

In [143]:
%matplotlib inline
import matplotlib.pyplot as plt
def plot_average_accross_n_projects_various_k_features(disgreement_mean_across__top_n_various_k, disgreement_mean_across__bottom_n_various_k, feature_n):

    feature_range = range(1, feature_n, 1)
    feature_agreement_top = disgreement_mean_across__top_n_various_k["feature_agreement"]
    rank_agreement_top = disgreement_mean_across__top_n_various_k["rank_agreement"]
    sign_agreement_top = disgreement_mean_across__top_n_various_k["sign_agreement"]
    signed_rank_agreement_top = disgreement_mean_across__top_n_various_k["signed_rank_agreement"]


    feature_agreement_bottom = disgreement_mean_across__bottom_n_various_k["feature_agreement"]
    rank_agreement_bottom = disgreement_mean_across__bottom_n_various_k["rank_agreement"]
    sign_agreement_bottom = disgreement_mean_across__bottom_n_various_k["sign_agreement"]
    signed_rank_agreement_bottom = disgreement_mean_across__bottom_n_various_k["signed_rank_agreement"]


    # Create separate subplots for each agreement metric
    fig, axs = plt.subplots(4, 1, figsize=(10, 16))

    # Plot Feature Agreement for top k
    axs[0].plot(feature_range, feature_agreement_top, label='Feature Agreement (Top 500)', marker='o', color='b')
    axs[0].plot(feature_range, feature_agreement_bottom, label='Feature Agreement (Bottom 500)', linestyle='--', marker='o', color='r')
    axs[0].set_xlabel('Number of Features (Top K)')
    axs[0].set_ylabel('Agreement Score')
    axs[0].set_title('Feature Agreement vs. Number of Features (Top K)')
    axs[0].legend()
    axs[0].grid(True)

    # Plot Rank Agreement for top k
    axs[1].plot(feature_range, rank_agreement_top, label='Rank Agreement (Top 500)', marker='s', color='g')
    axs[1].plot(feature_range, rank_agreement_bottom, label='Rank Agreement (Bottom 500)', linestyle='--', marker='s', color='m')
    axs[1].set_xlabel('Number of Features (Top K)')
    axs[1].set_ylabel('Agreement Score')
    axs[1].set_title('Rank Agreement vs. Number of Features (Top K)')
    axs[1].legend()
    axs[1].grid(True)

    # Plot Sign Agreement for top k
    axs[2].plot(feature_range, sign_agreement_top, label='Sign Agreement (Top 500)', marker='^', color='c')
    axs[2].plot(feature_range, sign_agreement_bottom, label='Sign Agreement (Bottom 500)', linestyle='--', marker='^', color='y')
    axs[2].set_xlabel('Number of Features (Top K)')
    axs[2].set_ylabel('Agreement Score')
    axs[2].set_title('Sign Agreement vs. Number of Features (Top K)')
    axs[2].legend()
    axs[2].grid(True)

    # Plot Signed Rank Agreement for top k
    axs[3].plot(feature_range, signed_rank_agreement_top, label='Signed Rank Agreement (Top 500)', marker='x', color='k')
    axs[3].plot(feature_range, signed_rank_agreement_bottom, label='Signed Rank Agreement (Bottom 500)', linestyle='--', marker='x', color='orange')
    axs[3].set_xlabel('Number of Features (Top K)')
    axs[3].set_ylabel('Agreement Score')
    axs[3].set_title('Signed Rank Agreement vs. Number of Features (Top K)')
    axs[3].legend()
    axs[3].grid(True)

    # Adjust spacing between subplots
    plt.tight_layout()

    # Show the plots
    plt.show()
#     plt.savefig("test.png")



In [144]:
disgreement_mean_across__top_500_various_k, disgreement_mean_across__bottom_500_various_k = disgreement_mean_across_top_bottom_500(global_explanation.values, explanations["fold0"]["lime"], 20)

In [145]:
disgreement_mean_across_top_bottom_500["feature_agreement"]

TypeError: 'function' object is not subscriptable

In [146]:
len(global_explanation)

575

In [ ]:
plot_average_accross_n_projects_various_k_features(disgreement_mean_across__top_500_various_k , disgreement_mean_across__bottom_500_various_k,20)

In [ ]:
disgreement_mean_across__top_500_various_k_max , disgreement_mean_across__bottom_500_various_max = disgreement_mean_across__top_500(explanations, 20, "max", raw_data)

In [ ]:
plot_average_accross_n_projects_various_k_features(disgreement_mean_across__top_500_various_k_max , disgreement_mean_across__bottom_500_various_max,20)

In [ ]:
disgreement_mean_across__top_500_various_k_max , disgreement_mean_across__bottom_500_various_max = disgreement_mean_across__top_500(explanations, 20, "activated", raw_data)
plot_average_accross_n_projects_various_k_features(disgreement_mean_across__top_500_various_k_max , disgreement_mean_across__bottom_500_various_max,20)

In [ ]:
disgreement_mean_across__top_500_various_k_max , disgreement_mean_across__bottom_500_various_max = disgreement_mean_across__top_500(explanations, 20, "maxnonactivated", raw_data)
plot_average_accross_n_projects_various_k_features(disgreement_mean_across__top_500_various_k_max , disgreement_mean_across__bottom_500_various_max,20)

## 20% features

In [ ]:
top , bottom = disgreement_mean_at_k(explanations, 4, method="max", raw_data=raw_data)


In [ ]:
print(top)

In [ ]:
print(bottom)